# Analyse scission - predefined model

Here we are going to assume that the scission rate is given by:
$$
k = \alpha |\mathrm{tr}(A \nabla U)|^\beta \mathrm{tr}(A)^\gamma
$$
and we will use available data to fit the model parameters.


In [1]:
import numpy as np
from matsindy.feature_library import FeatureLibrary

from ipywidgets import interactive, fixed
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.pyplot import cm
plt.rcParams['figure.figsize'] = [15, 8]

## Datasets selection

In [23]:
Wi_max = 1000                       # Peak Weissenberg number in simulations
n_links = 100                      # Number of segments in the Kramers chain
n_ensemble = 1000                     # Number of molecules to simulate
n_proc = 16                         # Number of processor cores to use
seq = np.random.SeedSequence(2022)  # Random number seed

# Input folder
data_folder = 'outputs/Kramers'
                    
# Name: file
scenarios = [
    'turbulence_0',
    'turbulence_1',
    'turbulence_2',
    'turbulence_3',
    'turbulence_4',
    'inkjet_0',
    'inkjet_1',
    'inkjet_2',
    'inkjet_3',
    'inkjet_4',
    'contraction_0',
    'contraction_1',
    'contraction_2',
    'contraction_3',
    'contraction_4',
    'contraction_5',
    'contraction_6',
    'elongation_0',
    'random_0',
    'random_1',
    'random_2',
    'random_3',
    'random_4',
    'sonication_0',
    'sonication_1',
    'sonication_2',
    'sonication_3',
    'sonication_4',
]
# Load data with rescaling
dataset = {}
for scenario in scenarios:
    with np.load(f"{data_folder}/{scenario}_Wi{Wi_max}_nlinks{n_links}_nmol{n_ensemble}.npz") as data:
        temp = dict(data)
        # Rename and normalise variables
        tau = 0.0142*n_links**2
        temp['A'] = temp.pop('A_average')/n_links**2
        temp['∇U'] = temp.pop('gradU')*tau
        temp['t'] = temp['t']/tau
        # Add transposed variables
        temp['∇Uᵀ'] = np.transpose(temp['∇U'], axes=(0, 2, 1))
        # Add features
        temp['tr(A)'] = np.trace(temp['A'], axis1=1, axis2=2)
        temp['tr(A∇U)'] = np.trace(temp['A']@temp['∇U'], axis1=1, axis2=2)
        temp['tr(A∇U∇Uᵀ)'] = np.trace(temp['A']@temp['∇U']@temp['∇Uᵀ'], axis1=1, axis2=2)        
        
        # Flow properties
        temp['epsilon_dot'] = np.linalg.eigvalsh(0.5*(temp['∇U'] + temp['∇Uᵀ']))
        
        # Effective principal strain rate
        temp['w'] = temp['tr(A∇U)']/temp['tr(A)']
        # Estimates of the Henky strain, one using principal strain rates, the other using polymer extension
        h = np.zeros_like(temp['t'])
        h_prime = np.zeros_like(temp['t'])
        for i, dt in enumerate(np.diff(temp['t'])):
            h[i+1] = (np.amax(temp['epsilon_dot'][i]) + h[i]/dt)/(1./dt + 1./1.)
            #h_prime[i+1] = (temp['tr(A∇U)'][i]/temp['tr(A)'][i] + h_prime[i]/dt)/(1./dt + 1./1.)
            h_prime[i+1] = temp['w'][i]*dt + h_prime[i]
        temp['h'] = h
        temp['h_prime'] = h_prime
        
        dgradU = np.zeros_like(temp['∇U'])
        dgradU[1:] = np.diff(temp['∇U'], axis=0)/np.diff(temp['t'])[:, None, None]
        temp['d∇U'] = dgradU
        temp['tr(Ad∇U)'] = np.trace(temp['A']@temp['d∇U'], axis1=1, axis2=2)


        # Save
        dataset[scenario] = temp

## Simulate scission

For an elongated chain, the maximum tension is at the centre and given by:
$$
g_\max = Wi\left(\frac{N^2}{8} + \frac{N}{2} \right)
$$
So a critical tension corresponds to a critical Weissenberg number
$$
Wi_c = \frac{g_c}{\frac{N^2}{8} + \frac{N}{2}}
$$

In [28]:
Wi_c =  100
threshold = Wi_c/(8*0.0142)
print(threshold)

for scenario, data in dataset.items():
    population = np.zeros_like(data['t'])
    for molecule in data['g_max']:
        for i, g in enumerate(molecule):
            if g < threshold:
                population[i] += 1
            else:
                break
    data['c'] = population/len(data['g_max'])
    k = np.zeros_like(data['t'])
    k[1:] = np.diff(-np.log(data['c']+1e-6))/np.diff(data['t'])
    data['k'] = k
    
    # Smooth k using decrete cosine formulation
    mirror = np.hstack((data['c'], np.flip(data['c'][1:])))
    p_hat = np.fft.rfft(mirror)
    f = np.fft.rfftfreq(len(mirror))
    filter_ = np.exp(-(5*f)**2)
    pop_smooth_m = np.fft.irfft(filter_*p_hat, n=len(mirror))
    pop_smooth = pop_smooth_m[:len(data['c'])]
    k_smooth = np.zeros_like(data['t'])
    k_smooth[1:] = np.diff(-np.log(pop_smooth+1e-6))/np.diff(data['t'])
    data['k_smooth'] = k_smooth
    data['c_smooth'] = pop_smooth
    
    ## Simulation based on infinity Wi model:
    w = data['w']/Wi_c*1.2
    alpha = (0.5 + np.log(w)**2/4)/(1 + np.exp(-(np.log(w))/0.1))
    beta = 0.5*np.log(n_links) - 0.22*np.log(w)
    gamma = 0.14

    minus_log_c = alpha*gamma*np.log((1+np.exp(-beta/gamma+(data['h_prime']+np.log(n_links)/12)/gamma)))
    data['c_model'] = np.exp(-minus_log_c)
    

880.2816901408451


/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in log
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in log
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in log


Inspect the result:

In [29]:
def view_scission(scenario, zoom):
    t = dataset[scenario]['t'][zoom[0]:zoom[1]]
    
    plt.rcParams['figure.figsize'] = [15, 15]
    fig, ax = plt.subplots(nrows=4)
    ax[0].plot(t, dataset[scenario]['c'][zoom[0]:zoom[1]])
    ax[0].plot(t, dataset[scenario]['c_model'][zoom[0]:zoom[1]], '--')
    ax[0].set_title('Population')
    
    gradU = dataset[scenario]['∇U'][zoom[0]:zoom[1]]
    D = 0.5*(gradU + np.transpose(gradU, axes=(0,2,1)))
    strains = np.linalg.eigvalsh(D)
    ax[1].plot(t, strains[:,0])
    ax[1].plot(t, strains[:,1])
    ax[1].plot(t, strains[:,2])
    ax[1].set_title('Strain rates')
    
    ax[2].semilogy(t, dataset[scenario]['tr(A)'][zoom[0]:zoom[1]]*dataset[scenario]['tr(Ad∇U)'][zoom[0]:zoom[1]]/(dataset[scenario]['tr(A∇U)'][zoom[0]:zoom[1]]**2+1e-6))
    ax[2].set_title('tr(Ad∇U)')
    
    ax[3].plot(t, dataset[scenario]['k'][zoom[0]:zoom[1]])
    ax[3].plot(t, dataset[scenario]['k_smooth'][zoom[0]:zoom[1]])
    ax[3].set_title('k')
    
    plt.show()

    
x_widget = widgets.Dropdown(options=list(dataset.keys()), value=list(dataset.keys())[0], description='Scenario:')
y_widget = widgets.IntRangeSlider(description='Zoom:', continuous_update=False)    

def update_models(*args):
    tmax = len(dataset[x_widget.value]['t'])
    y_widget.value=[0, tmax]
    y_widget.min=0
    y_widget.max=tmax
x_widget.observe(update_models)
    
ws = interactive(view_scission, 
                 scenario=x_widget,
                 zoom=y_widget
                )
ws

interactive(children=(Dropdown(description='Scenario:', options=('turbulence_0', 'turbulence_1', 'turbulence_2…

## Look for functional form

In [14]:
def view_3d(scenarios):
    #plt.rcParams['figure.figsize'] = [10, 8]
    #fig = plt.figure()
    #ax = plt.axes(projection='3d')
    #for scenario in scenarios:
    #    ax.scatter3D(dataset[scenario]['tr(A)'], dataset[scenario]['tr(A∇U)'], dataset[scenario]['k_smooth'])
    #ax.set_xlabel('tr(A)')
    #ax.set_ylabel('tr(A*gradU)')
    #ax.set_zlabel('k smooth')
    #plt.show()
    
    plt.rcParams['figure.figsize'] = [15, 5]
    #fig, ax = plt.subplots(ncols=2)
    for scenario in scenarios:
        plt.loglog(dataset[scenario]['tr(A∇U)'], dataset[scenario]['k_smooth'], '.', alpha=0.5)
        
        x = dataset[scenario]['tr(A∇U)']
        trA = dataset[scenario]['tr(A)']
        T = x > Wi_c
        y = 2*T*(x-Wi_c/n_links)**2/Wi_c/dataset[scenario]['tr(A)']
        plt.loglog(x, y, 'k.', alpha=0.2)
    plt.xlim(1, 2*Wi_max)
    plt.ylim(1e-1, 1e5)
    plt.grid()
    plt.show()
    
w3d = interactive(view_3d, 
                 scenarios=widgets.SelectMultiple(
                    options=list(dataset.keys()),
                    value=list(dataset.keys()),
                    #rows=10,
                    description='Scenarios:',
                    disabled=False
                    )
                )
w3d

interactive(children=(SelectMultiple(description='Scenarios:', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12…

## Optimise parameters

In [6]:
from scipy.optimize import minimize, shgo

fit_set = {
    'turbulence_0',
    'turbulence_1',
    'turbulence_2',
    'turbulence_3',
    'turbulence_4',
#    'inkjet_0',
#    'inkjet_1',
#    'inkjet_2',
#    'inkjet_3',
#    'inkjet_4',
    'contraction_0',
    'contraction_1',
    'contraction_2',
    'contraction_3',
    'contraction_4',
    'contraction_5',
    'contraction_6',
#    'elongation_0',
#    'random_0',
#    'random_1',
#    'random_2',
#    'random_3',
#    'random_4',
    'sonication_0',
    'sonication_1',
    'sonication_2',
    'sonication_3',
    'sonication_4',
}

def cost(params):
    total_cost = 0.
    for scenario in fit_set:
        T0 = Wi_c*dataset[scenario]['tr(A)']
        T = dataset[scenario]['tr(A∇U)'] > T0
        k = T*params[1]*(np.abs(dataset[scenario]['tr(A∇U)'])-T0)**2 *dataset[scenario]['tr(A)']**(-1)
        c_sim = np.ones_like(dataset[scenario]['c'])
        c_sim[1:] = np.exp(-np.cumsum(k[1:]*np.diff(dataset[scenario]['t'])))
        total_cost += np.average((dataset[scenario]['c']-c_sim)**2)
    return total_cost
        
res = minimize(cost, x0=[1.,1./Wi_c, 2, -1])
bounds = [(1,50), (0, 1), (1.5, 2.5)]
#res = shgo(cost, bounds, n=10000, iters=3)

/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: overflow encountered in exp
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in square
/home/etienne/miniconda3/lib/python3.7/site-packages/scipy/optimize/_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
/home/etienne/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: overflow encountered in square


In [7]:
res

      fun: 0.06449630074467914
 hess_inv: array([[1.       , 0.       , 0.       , 0.       ],
       [0.       , 0.0055273, 0.       , 0.       ],
       [0.       , 0.       , 1.       , 0.       ],
       [0.       , 0.       , 0.       , 1.       ]])
      jac: array([ 0.00000000e+00, -1.11758709e-08,  0.00000000e+00,  0.00000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 162
      nit: 5
     njev: 32
   status: 0
  success: True
        x: array([ 1.        ,  0.04447131,  2.        , -1.        ])

In [8]:
for scenario, data in dataset.items():
    T0 = Wi_c*dataset[scenario]['tr(A)']/res.x[0]
    T = dataset[scenario]['tr(A∇U)'] > T0
    k = T*res.x[1]*(np.abs(dataset[scenario]['tr(A∇U)'])-T0)**res.x[2] *dataset[scenario]['tr(A)']**res.x[3]
    c_sim = np.ones_like(data['c'])
    c_sim[1:] = np.exp(-np.cumsum(k[1:]*np.diff(data['t'])))
    data['c_sim'] = c_sim

In [9]:
def view_fit(scenario):
    plt.rcParams['figure.figsize'] = [15, 10]
    plt.plot(dataset[scenario]['t'], dataset[scenario]['c'], label='Molecular dynamics')
    plt.plot(dataset[scenario]['t'], dataset[scenario]['c_sim'], label='model')
    plt.axhline(0, ls=':')
    plt.legend()
    plt.show()

wf = interactive(view_fit, 
                 scenario=widgets.Dropdown(
                    options=list(dataset.keys()),
                    value=next(iter(dataset.keys())),
                    description='Scenario:',
                    disabled=False,
                    )
                )
wf

interactive(children=(Dropdown(description='Scenario:', options=('turbulence_0', 'turbulence_1', 'turbulence_2…

In [10]:
import scipy
scipy.__version__

'1.7.3'

In [11]:
2/50

0.04